In [2]:
%matplotlib inline
import matplotlib.pyplot as plt, seaborn as sn, emcee, corner
import inca
from inca_calibration import *
import numpy as np
from scipy.stats import norm
#from IPython import parallel
#c = parallel.Client()
#view = c.load_balanced_view()
sn.set_context('notebook')

In [3]:
### NOTE: This one is single-threaded. For a multithreaded version, see simplyp_emcee.py

def log_likelyhood(params, dataset, calibration, objective, obs):
    # NOTE: If we use a parallellized optimizer we need to make a copy of the dataset to not have several threads overwrite each other.
    # (in that case, only use the copy when setting parameter values, running the model, and extracting results below)
    datasetcopy = dataset.copy()
    
    set_values(datasetcopy, params, calibration)
    
    datasetcopy.run_model()
    
    fn, simname, simindexes, obsname, obsindexes, skiptimesteps = objective
    
    sim = datasetcopy.get_result_series(simname, simindexes)
    sim2 = sim[skiptimesteps:]
    obs2 = obs[skiptimesteps:]
    
    M = params[len(calibration)]
    sigma_e = M*sim2
    
    likes = norm(sim2, sigma_e).logpdf(obs2)
    
    like = np.nansum(likes)
    
    # NOTE: If we made a copy of the dataset we need to delete it so that we don't get a huge memory leak
    datasetcopy.delete()
    
    #print ('single evaluation. result: %f' % like)
    
    return like

In [4]:
def run_emcee2(dataset, min, max, initial_guess, calibration, objective, n_walk, n_steps, n_burn, truths=None,
              labels_short=None) :
    
    log_likelyhood, simname, simindexes, obsname, obsindexes, skiptimesteps = objective
    
    obs = dataset.get_input_series(obsname, obsindexes, alignwithresults=True)
    
    n_dim = len(initial_guess)
    
    def log_prior(params) :
        #for idx in range(len(params)):
        #    print('min:%f, max:%f, param:%f, range:%d' % 
        #          (min[idx], max[idx], params[idx], (params[idx] >= min[idx] and params[idx] <= max[idx])))
        
        if check_min_max(params, min, max) :
            return 0
        return -np.inf

    def log_posterior(params):
        log_pri = log_prior(params)
        
        if(np.isfinite(log_pri)):
            log_like = log_likelyhood(params, dataset, calibration, objective, obs)
            return log_pri + log_like
        return -np.inf

    starting_guesses = [initial_guess + 1e-4*np.random.randn(n_dim) for i in range(n_walk)]

    sampler = emcee.EnsembleSampler(n_walk, n_dim, log_posterior)#, pool=view)
    
    pos, prob, state = sampler.run_mcmc(starting_guesses, n_steps)
    
    print('\nAverage acceptance rate: %f' % np.mean(sampler.acceptance_fraction))

    
    param_labels = ['%s [%s] (%s)' % (cal[0], ', '.join(cal[1]), dataset.get_parameter_unit(cal[0])) 
                    for cal in calibration]
    param_labels.append('M')
    
    fig, axes = plt.subplots(nrows=n_dim, ncols=1, figsize=(10, 20))    
    for idx, title in enumerate(param_labels):        
        axes[idx].plot(sampler.chain[:,:,idx].T, '-', color='k', alpha=0.3)
        axes[idx].set_title(title, fontsize=16) 
    plt.subplots_adjust(hspace=0.7)    
    plt.show()
    
    samples = sampler.chain[:, n_burn:, :].reshape((-1, n_dim))
    
    # Triangle plot
    if labels_short != None:
        labels = labels_short
    else: labels = param_labels
    tri = corner.corner(samples,
                        labels=labels,
                        truths=truths,
                        quantiles=[0.025, 0.5, 0.975],
                        show_titles=True, 
                        title_args={'fontsize': 24},
                        label_kwargs={'fontsize': 20})
    
    return samples

In [5]:
inca.initialize('simplyp.dll')

dataset = inca.DataSet.setup_from_parameter_and_input_files('optimal_parameters.dat',
                                                            '../Applications/SimplyP/tarlandinputs.dat')

calibration = [
    ('Proportion of precipitation that contributes to quick flow', ['Tarland1']),
    ('Baseflow index',                                             ['Tarland1']),
    ('Groundwater time constant',                                  ['Tarland1']),
    ('Gradient of stream velocity-discharge relationship',         ['Tarland1']),
    ('Exponent of stream velocity-discharge relationship',         ['Tarland1']),
    ('Soil water time constant',                                   ['Arable']),
    ('Soil water time constant',                                   ['Semi-natural']),
    ]
    
labels_short = [r'$f_{quick}$', r'$\beta$', r'$T_g$', r'$a$', r'$b$', r'$T_s[A]$', r'$T_s[S]$', r'M']

#NOTE: This reads the initial guess that was provided by the parameter file.
#initial_guess = default_initial_guess(dataset, calibration)
#initial_guess.append(0.243586) # This is the initial M.

#min = [0.25 * x for x in initial_guess]
#max = [4.0 * x for x in initial_guess]

#NOTE: Constrain to the min and max values recommended by the model in case we made our bounds too wide.
#constrain_min_max(dataset, calibration, min, max)

initial_guess = [0.04, 0.53, 179.78, 0.2,  0.34, 5.23, 19.47, 0.23]
min =           [0.01, 0.45,  90,    0.05, 0.1,  3,    10,    0.19]
max =           [0.08, 0.65, 240,    0.8,  0.9, 15,    40,    0.26]

skiptimesteps = 50   # Skip these many of the first timesteps in the objective evaluation

objective = (log_likelyhood, 'Daily mean reach flow', ['Tarland1'], 'observed Q mm/d', [], skiptimesteps)

samples = run_emcee2(dataset, min, max, initial_guess, calibration, objective, n_walk=20, n_steps=50, n_burn=0,
                    labels_short=labels_short)

WindowsError: exception: access violation reading 0x0000000000000020

In [ ]:
dataset.delete()